<a href="https://colab.research.google.com/github/dokeyuka/deep-learning-practice/blob/main/DL_practice_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 時系列データ
## オーバーフローを起こさないために
## 重みの初期化：直行行列
## 活性化関数：ReLUではなくtanh
## を用いる

In [1]:


def sin(x, T = 100):
  return np.sin(2.0*np.pi*x/T)

def toy_problem(T = 100, ampl = 0.05):
  x = np.arange(0, 2*T+1)
  noise = ampl*np.random.uniform(low = -1.0, high = 1.0, size = len(x))
  return sin(x)+noise

T = 100
f = toy_problem(T).astype(np.float32)

length_of_sequences = len(f)#2T+1

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import os
import numpy as np
from sklearn.metrics import accuracy_score
import torch.optim as optimizers
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
#from callbacks import EarlyStopping



class RNN(nn.Module):
  def __init__(self, hidden_dim):
    super().__init__()
    # batch_first = Trueで(データ数N,　時系列長tau,　入力次元l)の順にデータを与える
    self.l1 = nn.RNN(1, hidden_dim, 
                     nonlinearity = 'tanh',
                     batch_first = True)
    
    self.l2 = nn.Linear(hidden_dim, 1)
    
    nn.init.xavier_normal_(self.l1.weight_ih_l0)#入力層-隠れ層(input-hidden)の重みの初期化,l0:一つ目の隠れ層
    nn.init.orthogonal_(self.l1.weight_hh_l0)#隠れ層-隠れ層(hidden-hidden)の重みの初期化

  def forward(self, x):
    #戻り値(各時刻における隠れ層の出力、隠れ層の値)
    h, _ = self.l1(x)
    #出力層には再gの時刻の隠れ層の出力を与える
    y = self.l2(h[:,-1])

    return y

'''
1 modelの構築
'''
model = RNN(50).to(device)


maxlen = 25
x = []
t = []
for i in range(length_of_sequences-maxlen):
  x.append(f[i:i+maxlen])
  t.append(f[i+maxlen])

x = np.array(x).reshape(-1,maxlen,1)
t = np.array(t).reshape(-1,1)

x_train, x_val, t_train, t_val = train_test_split(x,t,test_size = 0.2, shuffle = False)

ModuleNotFoundError: ignored